In this notebook, we will look at how to use spark-sklearn project to run distributed cross validation of the Scikit-learn models. 

Since Scikit-learn models are by design not distributed, this will help us use spark cluster's several cpu resources to run cross validation. This is similar to the scikit-learn's joblib for multicore parallelism - https://pythonhosted.org/joblib/

In [1]:
print sc
print sqlContext
print sqlCtx

Run a RandomForestClassifier on the digits dataset in Scikit-learn and use Scikit-learn's GridSearch cross validation to fit the best model. 

This is a bit slower as the cross validation is happening on a single server

In [4]:
from sklearn import grid_search, datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
digits = datasets.load_digits()
X, y = digits.data, digits.target
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"],
              "n_estimators": [10, 20, 40, 80]}
gs = grid_search.GridSearchCV(RandomForestClassifier(), param_grid=param_grid)
gs.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'bootstrap': [True, False], 'min_samples_leaf': [1, 3, 10], 'n_estimators': [10, 20, 40, 80], 'min_samples_split': [1, 3, 10], 'criterion': ['gini', 'entropy'], 'max_features': [1, 3, 10], 'max_depth': [3, None]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

We now will import the spark-sklearn package and run scikit-learn's RandomForestClassifier on digits dataset and use Spark-sklearn's GridSearchCV cross validation to fit the best model

This will help you leverage multiple cores and memory available on the large spark cluster to run grid search on a very large parameter grid. This will also run faster compared to the single thread

As you can see the only change in that we need to import the GridSearchCV from spark_learn instead of sklear.grid_search.

In [5]:
from sklearn import grid_search, datasets
from sklearn.ensemble import RandomForestClassifier
# from sklearn.grid_search import GridSearchCV
# Use spark_sklearn’s grid search instead:
from spark_sklearn import GridSearchCV
digits = datasets.load_digits()
X, y = digits.data, digits.target
param_grid = {"max_depth": [3, None],
              "max_features": [1, 3, 10],
              "min_samples_split": [1, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"],
              "n_estimators": [10, 20, 40, 80]}
gs = grid_search.GridSearchCV(RandomForestClassifier(), param_grid=param_grid)
gs.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'bootstrap': [True, False], 'min_samples_leaf': [1, 3, 10], 'n_estimators': [10, 20, 40, 80], 'min_samples_split': [1, 3, 10], 'criterion': ['gini', 'entropy'], 'max_features': [1, 3, 10], 'max_depth': [3, None]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

On top of parallelizing scikit-learns cross validation, Spark-sklearn has some other features like converting between scikit-learn models and spark mllib models.

More on this here.

http://pythonhosted.org/spark-sklearn/
